# Pseudo-code for project 2

OBJECTIVE: Find parameters of shots that lead to goals using exhaustive parameter scan

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

Should be at least 3 pages long

First, define a shooting alogthim function in terms of 

    rho = air density
    
    g = Accl due to gravity
    
    m = mass 
    
    D = diameter 
    
    v0 = speed 
    
    omega = spin (the axis of rotation is perpentidular with the ground)
    
    phi = azimuthal angle (horivantal angle)
    
    Theta = elevation angle (vertical angle)
    
[inspiration can be taken from the baseball homework, hw 7]

Second, set up enviroemnts with proper dimensions for the two different types of kicks, including the goal and its target area. 
    1. free kick 
    2. corner kick

Third, run shooting alogthim through range of paramters. 

In [2]:
#parameters for free kick 
rho = 1.2     # kg m^-3
g= 9.81     # kg m s^-2
m = 0.436    # kg
D = 0.218     # m
v0 = np.arange(24, 30.1, .1)   # m s^-1
omega = np.arange(0, 75.6, .2)   # rad s^-1
phi = np.arange(-0.0349066, 0.0872665, .001)  # rad [-2 deg to 5 deg]
theta = np.arange(.174533, 0.349066, .001) # rad [10 deg to 20 deg]

In [3]:
#parameters for corner kick
rho = 1.2     # kg m^-3
g= 9.81     # kg m s^-2
m = 0.436    # kg
D = 0.218     # m
v0 = np.arange(24, 30.1, .1)   # m s^-1
omega = np.arange(0, 75.6, .2)   # rad s^-1
phi = np.arange(-0.0698132, 0.0698132, .001)  # rad [-4 deg to 4 deg]
theta = np.arange(0.349066, 0.523599, .001) # rad [20 deg to 30 deg]

Objectives
 1. talk about drag and lift coeffcients of ball
 2. simulate trajectories of free kick and corner kick
 3. plot in 2D and 3D and desricbe paramters of trajectories
 4. Talk in depth about at one goal scoring shot for each: include parameters plots and a plot where omega = 0 to show influence of spin.

trajectory of free kick should stop calculating when
  1. ball reaches goal line
  2. ball touches ground
  3. ball touches wall

trajectory of corner kick should stop calculating when
  1. ball reaches goal line
  2. ball touches ground

Possbily use 3D-plotting to show trajectories of shots. (especally goal-making shots). guidense can come from 14_PDEs_2.ipynb

include refences and acknolegdments 
